# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import matplotlib as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()
import re

%matplotlib inline

In [4]:
sns.set_theme(
    style = 'whitegrid'
    
    



)

In [5]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []
stars = []
country= []
date= []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
        
    for para in parsed_content.find_all("div", class_ = "rating-10"):
        stars.append(para.span.get_text())
        
    for para in parsed_content.find_all("h3"):
        country.append(para.span.next_sibling.get_text().strip("( )"))
        
        
    for para in parsed_content.find_all('time'):
        date.append(para.get_text())
                
  
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [6]:
len(reviews)

len(country)
len(date)


1000

In [7]:
df = pd.DataFrame({"reviews":reviews,"stars": stars[0:1000], "date":date, "country": country})

In [8]:
df

,reviews,stars,date,country
0,✅ Trip Verified | We were traveling as a fami...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,13th June 2023,United States
1,✅ Trip Verified | Flight at 8.40am from DUB to...,3,12th June 2023,Australia
2,✅ Trip Verified | Terrible. I have traveled t...,1,11th June 2023,United Kingdom
3,✅ Trip Verified | The customer service is ugl...,1,11th June 2023,United States
4,✅ Trip Verified | Most uncomfortable flight I...,1,10th June 2023,United Kingdom
...,...,...,...,...
995,✅ Trip Verified | Istanbul to London Heathrow...,8,6th August 2018,United Kingdom
996,✅ Trip Verified | Bad service and attitude al...,7,3rd August 2018,United Kingdom
997,✅ Trip Verified | London to Santiago de Chile...,10,31st July 2018,Germany
998,✅ Trip Verified | Flew a now ageing B777 flig...,7,30th July 2018,United Kingdom


In [9]:


df['stars'] = df['stars'].str.strip('\n \t')


In [10]:
df.date= pd.to_datetime(df.date)

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [11]:
df.dtypes

reviews            object
stars              object
date       datetime64[ns]
country            object
dtype: object

In [12]:
df['verify']  =df['reviews'].str.contains("Trip Verified")

In [13]:
df['reviews'] = df['reviews'].str.strip('✅ Trip Verified |')
df['reviews'] = df['reviews'].str.strip('❎ Not Verified |')

In [15]:
df

,reviews,stars,date,country,verify
0,We were traveling as a family (5 people). Beca...,5,2023-06-13,United States,True
1,Flight at 8.40am from DUB to LCY cancelled 115...,3,2023-06-12,Australia,True
2,ble. I have traveled twice with them this year...,1,2023-06-11,United Kingdom,True
3,he customer service is ugly. Tried calling two...,1,2023-06-11,United States,True
4,Most uncomfortable flight I have ever experien...,1,2023-06-10,United Kingdom,True
...,...,...,...,...,...
995,Istanbul to London Heathrow. The plane itself ...,8,2018-08-06,United Kingdom,True
996,Bad service and attitude all round. My fiasco ...,7,2018-08-03,United Kingdom,True
997,London to Santiago de Chile. I was flying prem...,10,2018-07-31,Germany,True
998,Flew a now ageing B777 flight BA 0053 to Seatt...,7,2018-07-30,United Kingdom,True


In [16]:
for i in df['reviews']:

    i = re.sub('[^a-zA-Z]',' ', i)
    i = i.lower()
    i = i.split()
    i = [lemma.lemmatize(word) for word in i if word not in set(stopwords.words("english"))]
    i = " ".join(i)
    


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/abolfazl/nltk_data'
    - '/home/abolfazl/anaconda3/envs/britishairway/nltk_data'
    - '/home/abolfazl/anaconda3/envs/britishairway/share/nltk_data'
    - '/home/abolfazl/anaconda3/envs/britishairway/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [45]:
df['reviews'] = df['reviews'].apply(lambda x :  gettxt(x))

In [47]:
df

,reviews
0,We were traveling as a family (5 people). Beca...
1,Flight at 8.40am from DUB to LCY cancelled 115...
2,Terrible. I have traveled twice with them this...
3,The customer service is ugly. Tried calling tw...
4,Most uncomfortable flight I have ever experien...
...,...
995,Istanbul to London Heathrow. The plane itself ...
996,Bad service and attitude all round. My fiasco ...
997,London to Santiago de Chile. I was flying prem...
998,Flew a now ageing B777 flight BA 0053 to Seatt...


In [54]:
def getverify(x):
        
        text =  x.split('|')[-1]
        return text 
    

In [55]:
df['reviews'] = df['reviews'].apply(lambda x: getverify(x))


In [60]:
df

,reviews
0,Absolutely horrible experience. I booked a t...
1,This is the worst airline. Not one thing wen...
2,I will never fly British Airways again. To s...
3,Worst aircraft I have ever flown. The seats ...
4,I enjoyed my flight. The boarding was swift ...
...,...
995,London to Zurich. The airline isn’t bad. The...
996,Glasgow to New York JFK via London. Outgoing...
997,Flew on a last minute emergency trip London ...
998,"Earlier this year, my wife and I travelled b..."


In [67]:
text = "The film Titanic was released in 1998"

In [68]:
import re 

In [71]:
f = re.match(r"film", text)

In [72]:
f

In [81]:
result = re.match(r"[a-zA-z]+", text)

In [82]:
type (result)

re.Match

In [76]:
type(f)

NoneType

In [87]:
result.group(1)

IndexError: no such group